In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [2]:
from keras.datasets import mnist
from keras import models
from keras import layers
from keras import initializers
from keras import optimizers

In [3]:
batch_size = 32
num_classes = 10
# epochs = 200
epochs = 5
hidden_units = 100

learning_rate = 1e-6
clip_norm = 1.0

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], -1, 1)
x_test = x_test.reshape(x_test.shape[0], -1, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 784, 1)
60000 train samples
10000 test samples


In [5]:
print('Evaluate IRNN...')
model = models.Sequential()
model.add(layers.SimpleRNN(hidden_units,
                           kernel_initializer=initializers.RandomNormal(stddev=0.001),
                           recurrent_initializer=initializers.Identity(gain=1.0),
                           activation='relu',
                           input_shape=x_train.shape[1:]))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))
model.summary()
rmsprop = optimizers.RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

Evaluate IRNN...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 11,210
Trainable params: 11,210
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 590s 10ms/step - loss: 2.1024 - acc: 0.2065 - val_loss: 1.9957 - val_acc: 0.2448
Epoch 2/5
60000/60000 [==============================] - 588s 10ms/step - loss: 1.9519 - acc: 0.2554 - val_loss: 1.9162 - val_acc: 0.2800
Epoch 3/5
60000/60000 [==============================] - 591s 10ms/step - loss: 1.9101 - acc: 0.2773 - val_loss: 1.8873 - val_acc: 0.3277
Epoch 4/5
60000/60000 [==============================] - 591s 10ms/step - loss: 1.8724 - acc: 0.2972 - val_loss: 1.8416 - val_acc: 0.3243
Epoch 5/5
60000/60000 [==============================] - 589s 10ms/step - loss: 1.8311 - acc: 0.3217 - val_loss: 1.7865 - val_acc: 0.3564


In [7]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('IRNN test score:', scores[0])
print('IRNN test accuracy:', scores[1])

IRNN test score: 1.7864595052719117
IRNN test accuracy: 0.3564
